In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np

2021-10-06 16:30:33.387556: I tensorflow/stream_executor/platform/default/dso_loader.cc:54] Successfully opened dynamic library libcudart.so.11.0


In [3]:
img_size = (512, 512)

In [3]:
# inputs = keras.Input(shape=img_size + (1,))
# x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
# x = layers.BatchNormalization()(x)
# x = layers.Activation("relu")(x)

In [8]:
class Block(tf.keras.layers.Layer):
    def __init__(self, in_ch, out_ch, name=None):
        super(Block, self).__init__(name=name)
        self.conv1 = layers.Conv2D(in_ch, out_ch, 3, padding='same')
        self.relu1 = layers.ReLU()
        self.bn1   = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(out_ch, out_ch, 3, padding = 'same')
        self.relu2 = layers.ReLU()
        self.bn2   = layers.BatchNormalization()
#         self.w = tf.Variable(
#           tf.random.normal([input_dim, output_size]), name='w')
#         self.b = tf.Variable(tf.zeros([output_size]), name='b')
    def call(self, x):
        return self.bn2(self.relu2(self.conv2(self.bn1(self.relu1(self.conv1(x))))))

In [5]:
inputs = keras.Input(shape=img_size + (1,))
Block(1, 32)(inputs)

2021-10-06 15:24:04.016663: I tensorflow/stream_executor/platform/default/dso_loader.cc:54] Successfully opened dynamic library libcuda.so.1
2021-10-06 15:24:06.647498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-06 15:24:06.648793: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1734] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-10-06 15:24:06.649052: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-06 15:24:06.650248: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1734] Found device 1 with properties: 
pciBusID: 0000:00:05.0 na

<KerasTensor: shape=(None, 57, 57, 32) dtype=float32 (created by layer 'block')>

In [4]:
inputs = keras.Input(shape=img_size + (1,))

In [5]:
inputs

<KerasTensor: shape=(None, 512, 512, 1) dtype=float32 (created by layer 'input_1')>

In [6]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, chs=(1,32,64,128,256)):
        super().__init__()
        self.enc_blocks = [Block(chs[i], chs[i+1]) for i in range(len(chs)-1)]
        self.pool       = layers.MaxPool2D((2,2), padding='same')
    
    def call(self, x):
        ftrs = []
        for block in self.enc_blocks:
            x = block(x)
            ftrs.append(x)
            x = self.pool(x)
        return ftrs

In [ ]:
Encoder((1,32,64,128,256))(inputs)

2021-10-06 16:31:21.322892: I tensorflow/stream_executor/platform/default/dso_loader.cc:54] Successfully opened dynamic library libcuda.so.1
2021-10-06 16:31:23.994233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-06 16:31:23.995137: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1734] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-10-06 16:31:23.995269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-06 16:31:23.996112: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1734] Found device 1 with properties: 
pciBusID: 0000:00:05.0 na

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, chs=(256, 128, 64, 32)):
        super().__init__()
        self.chs         = chs
        self.upconvs    = [layers.Conv2DTranspose(chs[i], chs[i+1], 2, 'same') for i in range(len(chs)-1)]
        self.dec_blocks = [Block(chs[i], chs[i+1]) for i in range(len(chs)-1)]
        
    def call(self, x, encoder_features):
        for i in range(len(self.chs)-1):
            x        = self.upconvs[i](x)
#            enc_ftrs = self.crop(encoder_features[i], x)
#            x        = torch.cat([x, enc_ftrs], dim=1)
            x        = tf.concat([x, encoder_features[i]], dim=1)
            x        = self.dec_blocks[i](x)
        return x

In [ ]:
Decoder((256, 128, 64, 32))

In [ ]:
class Unet(tf.keras.layers.Layer):
    def __init__(self, enc_chs=[1,32,64,128,256], dec_chs=[256, 128, 64, 32], num_class=1, retain_dim=False, out_sz=(512,512)):
        super().__init__()
        self.encoder     = Encoder(enc_chs)
        self.decoder     = Decoder(dec_chs)
        self.head        = layers.Conv2D(dec_chs[-1], num_class, 1)
        self.retain_dim  = retain_dim

    def call(self, x):
        enc_ftrs = self.encoder(x)
        out      = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        out      = self.head(out)
        if self.retain_dim:
            out = F.interpolate(out, out_sz)
        return out

In [ ]:
inputs = keras.Input(shape=img_size + (1,))
model = Unet()(inputs)

In [ ]:
model.summary()

In [61]:
model.submodules

(<__main__.Decoder at 0x7f8db193a8b0>,
 <tensorflow.python.keras.layers.advanced_activations.ReLU at 0x7f8da002d4c0>)

In [62]:
import os

input_dir = "/data/small/train/image/"
target_dir = "/data/small/train/mask/"
img_size = (512, 512)
num_classes = 2 # change this
batch_size = 12 # changed to 12 from 32

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".npy")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".npy") and not fname.startswith(".")
    ]
)

print("Number of samples:", len(input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)

Number of samples: 2544
/data/small/train/image/0-0.npy | /data/small/train/mask/0-0.npy
/data/small/train/image/0-1.npy | /data/small/train/mask/0-1.npy
/data/small/train/image/0-10.npy | /data/small/train/mask/0-10.npy
/data/small/train/image/0-11.npy | /data/small/train/mask/0-11.npy
/data/small/train/image/0-12.npy | /data/small/train/mask/0-12.npy
/data/small/train/image/0-13.npy | /data/small/train/mask/0-13.npy
/data/small/train/image/0-14.npy | /data/small/train/mask/0-14.npy
/data/small/train/image/0-15.npy | /data/small/train/mask/0-15.npy
/data/small/train/image/0-16.npy | /data/small/train/mask/0-16.npy
/data/small/train/image/0-17.npy | /data/small/train/mask/0-17.npy


In [63]:
def load_image(imagepath, maskpath):
    image = np.load(imagepath).astype('float32')
#     image -= np.mean(image)
#     image /= np.std(image)
    # we will encounter divide by zero
    image = ((image - (image.min()+1)) * (1/((image.max() - image.min() + 1)) * 255)).astype('uint8')
    image = image.astype('float32')
    mask = np.load(maskpath)
    return image, mask

In [64]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img


class OxfordPets(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            #img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            img, msk = load_image(path, path)
            #x[j] = np.expand_dims(img, 2)
            x[j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            #img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            img, msk = load_image(path, path)
            #y[j] = np.expand_dims(img, 2)
            y[j] = msk
            # Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
            y[j] -= 1
            y1 = y[j] < 1
            y[j] = y1.astype(int)
        return x, y

In [65]:
import random

# Split our img paths into a training and a validation set
val_samples = 1000
idxs = np.arange(len(input_img_paths))
random.Random(1337).shuffle(idxs)
random.Random(1337).shuffle(idxs)
train_input_img_paths = np.array(input_img_paths)[idxs][:-val_samples]
train_target_img_paths = np.array(target_img_paths)[idxs][:-val_samples]
val_input_img_paths = np.array(input_img_paths)[idxs][-val_samples:]
val_target_img_paths = np.array(target_img_paths)[idxs][-val_samples:]

# Instantiate data Sequences for each split
train_gen = OxfordPets(
    batch_size, img_size, train_input_img_paths, train_target_img_paths
)
val_gen = OxfordPets(batch_size, img_size, val_input_img_paths, val_target_img_paths)

In [67]:
inp, tar = train_gen[0]

In [69]:
inp.shape

(12, 512, 512, 1)

In [70]:
model(inp[0])

TypeError: 'Unet' object is not callable